## Reference Material

* [Hudi Documentation Quick Start Guide][1]
* [EMR Hudi Documentation][2]
* [EMR Hudi Docuementation - Work with a Hudi dataset][3]

[1]:https://hudi.apache.org/docs/quick-start-guide/#setup]
[2]:https://docs.aws.amazon.com/emr/latest/ReleaseGuide/emr-hudi.html
[3]:https://docs.aws.amazon.com/emr/latest/ReleaseGuide/emr-hudi-work-with-dataset.html

## Configuration

Before running the code in the cell(s) below SSH into your EMR cluster and run the following 

```hdfs dfs -mkdir -p /apps/hudi/lib```

```hdfs dfs -copyFromLocal /usr/lib/hudi/hudi-spark-bundle.jar /apps/hudi/lib/hudi-spark-bundle.jar```

```hdfs dfs -copyFromLocal /usr/lib/spark/external/lib/spark-avro.jar /apps/hudi/lib/spark-avro.jar```

This will copy the Hudi jar files from the local file system to HDFS on the master node of the notebook cluster

In [1]:
%%configure
{
    "conf": {
            "spark.jars":"hdfs:///apps/hudi/lib/hudi-spark-bundle.jar,hdfs:///apps/hudi/lib/spark-avro.jar",
            "spark.serializer":"org.apache.spark.serializer.KryoSerializer",
            "spark.sql.hive.convertMetastoreParquet":"false"
    }
}

In [2]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

from datetime import datetime

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
2,application_1636114111194_0002,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Create a DataFrame

In [3]:
data = [
        ("1", "Chris", "2020-01-01", datetime.strptime('2020-01-01 00:00:00', '%Y-%m-%d %H:%M:%S')),
        ("2", "Will", "2020-01-01", datetime.strptime('2020-01-01 00:00:00', '%Y-%m-%d %H:%M:%S')),
        ("3", "Emma", "2020-01-01", datetime.strptime('2020-01-01 00:00:00', '%Y-%m-%d %H:%M:%S')),
        ("4", "John", "2020-01-01", datetime.strptime('2020-01-01 00:00:00', '%Y-%m-%d %H:%M:%S')),
        ("5", "Eric", "2020-01-01", datetime.strptime('2020-01-01 00:00:00', '%Y-%m-%d %H:%M:%S')),
        ("6", "Adam", "2020-01-01", datetime.strptime('2020-01-01 00:00:00', '%Y-%m-%d %H:%M:%S'))
]

schema = StructType([
        StructField("id", StringType(), False),
        StructField("name", StringType(), False), 
        StructField("create_date", StringType(), False),             
        StructField("last_update_time", TimestampType(), False)
])

inputDF = spark.createDataFrame(data=data,schema=schema)

# inputDF.show()
# inputDF.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Write to S3 via. Hudi

We create a ```hudiOptions``` variable. We use this when we write data to S3. 

DataSourceWriteOptions for ***Hudi***: 

Option|Description
:---|:---|
TABLE_NAME|The table name under which to register the dataset
TABLE_TYPE_OPT_KEY|Optional. Specifies whether the dataset is created as ```COPY_ON_WRITE``` or ```MERGE_ON_READ```. The default is ```COPY_ON_WRITE```
RECORDKEY_FIELD_OPT_KEY|The record key field whose value will be used as the recordKey component of HoodieKey. Actual value will be obtained by invoking .toString() on the field value. Nested fields can be specified using the dot notation, for example, a.b.c
PARTITIONPATH_FIELD_OPT_KEY|The partition path field whose value will be used as the partitionPath component of HoodieKey. The actual value will be obtained by invoking .toString() on the field value
PRECOMBINE_FIELD_OPT_KEY|The field used in pre-combining before actual write. When two records have the same key value, Hudi picks the one with the largest value for the precombine field as determined by Object.compareTo(..)

DataSourceWriteOptions for ***Hive***:

Option|Description
:---|:---|
HIVE_DATABASE_OPT_KEY|The Hive database to sync to. The default is ```default```
HIVE_PARTITION_EXTRACTOR_CLASS_OPT_KEY|The class used to extract partition field values into Hive partition columns
HIVE_PARTITION_FIELDS_OPT_KEY|The field in the dataset to use for determining Hive partition columns
HIVE_SYNC_ENABLED_OPT_KEY|When set to ```true```, registers the dataset with the Apache Hive metastore.                     
HIVE_TABLE_OPT_KEY|Required. The name of the table in Hive to sync to. For example the table name can be, ```my_hudi_table``` or any other name specified
HIVE_USER_OPT_KEY|Optional. The Hive user name to use when syncing. For example, ```hadoop```
HIVE_PASS_OPT_KEY|Optional. The Hive password for the user specified by HIVE_USER_OPT_KEY
HIVE_URL_OPT_KEY|The Hive metastore URL

*ensure that the ```hoodie.datasource.write.partitionpath.field``` and ```hoodie.datasource.write.precombine.field``` use different columns*

In [4]:
# Create hudiOptions variable
hudiOptions = {
    'hoodie.table.name': 'my_hudi_table',
    'hoodie.datasource.write.recordkey.field': 'id',
    'hoodie.datasource.write.partitionpath.field': 'create_date',
    'hoodie.datasource.write.precombine.field': 'last_update_time',
    'hoodie.datasource.hive_sync.enable': 'true',
    'hoodie.datasource.hive_sync.table': 'my_hudi_table',
    'hoodie.datasource.hive_sync.partition_fields': 'last_update_time',
    'hoodie.datasource.hive_sync.partition_extractor_class': 'org.apache.hudi.hive.MultiPartKeysValueExtractor'
}

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

*Note* adjust the s3 path in ```.save()```

Options for ```hoodie.datasource.write.operation```

Option|Description
:---|:---|
UPSERT|This is the default operation where the input records are first tagged as inserts or updates by looking up the index. The records are ultimately written after heuristics are run to determine how best to pack them on storage to optimize for things like file sizing. This operation is recommended for use-cases like database change capture where the input almost certainly contains updates. The target table will never show duplicates
INSERT|This operation is very similar to upsert in terms of heuristics/file sizing but completely skips the index lookup step. Thus, it can be a lot faster than upserts for use-cases like log de-duplication (in conjunction with options to filter duplicates mentioned below). This is also suitable for use-cases where the table can tolerate duplicates, but just need the transactional writes/incremental pull/storage management capabilities of Hudi
BULK_INSERT|Both upsert and insert operations keep input records in memory to speed up storage heuristics computations faster (among other things) and thus can be cumbersome for initial loading/bootstrapping a Hudi table at first. Bulk insert provides the same semantics as insert, while implementing a sort-based data writing algorithm, which can scale very well for several hundred TBs of initial load. However, this just does a best-effort job at sizing files vs guaranteeing file sizes like inserts/upserts do

In [5]:
# Write a DataFrame to S3 as a Hudi dataset 
inputDF \
    .write \
    .format('org.apache.hudi') \
    .option('hoodie.datasource.write.operation', 'insert') \
    .options(**hudiOptions) \
    .mode('overwrite') \
    .save('s3://hudi-sharkech/myhudidataset/')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Read the Hudi Table

Hudi performs snapshot queries by default. Snapshot queries retrieve data at the present point in time

In [6]:
snapshotQueryDF = spark.read.format('org.apache.hudi').load('s3://hudi-sharkech/myhudidataset' + '/*/*')

# snapshotQueryDF.orderBy("id").show()
# snapshotQueryDF.select("_hoodie_commit_time", "_hoodie_commit_seqno", "_hoodie_record_key", "_hoodie_partition_path", "_hoodie_file_name").orderBy("_hoodie_record_key").show()

snapshotQueryDF.select("id", "name", "create_date", "last_update_time").orderBy("id").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---+-----+-----------+-------------------+
| id| name|create_date|   last_update_time|
+---+-----+-----------+-------------------+
|  1|Chris| 2020-01-01|2020-01-01 00:00:00|
|  2| Will| 2020-01-01|2020-01-01 00:00:00|
|  3| Emma| 2020-01-01|2020-01-01 00:00:00|
|  4| John| 2020-01-01|2020-01-01 00:00:00|
|  5| Eric| 2020-01-01|2020-01-01 00:00:00|
|  6| Adam| 2020-01-01|2020-01-01 00:00:00|
+---+-----+-----------+-------------------+

## Upsert data

Lets do an upsert ...

In [7]:
# Create a new data frame with an updated last_update_time
data = [
        ("1", "Chris Sharkey", "2020-01-01", datetime.strptime('2020-01-02 00:00:00', '%Y-%m-%d %H:%M:%S')),
        ("7", "Kelly", "2020-01-02", datetime.strptime('2020-01-02 00:00:00', '%Y-%m-%d %H:%M:%S'))
]

schema = StructType([
        StructField("id", StringType(), False),
        StructField("name", StringType(), False),
        StructField("create_date", StringType(), False),             
        StructField("last_update_time", TimestampType(), False)
])

updateDF = spark.createDataFrame(data=data,schema=schema)

# updateDF.show()
# updateDF.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
# Upsert the records in updateDF
updateDF \
    .write \
    .format('org.apache.hudi') \
    .option('hoodie.datasource.write.operation', 'upsert') \
    .options(**hudiOptions) \
    .mode('append') \
    .save('s3://hudi-sharkech/myhudidataset/')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
# Check that the upsert worked
snapshotQueryDF = spark.read.format('org.apache.hudi').load('s3://hudi-sharkech/myhudidataset' + '/*/*')

# snapshotQueryDF.show()

snapshotQueryDF.select("id", "name", "create_date", "last_update_time").orderBy("id").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---+-------------+-----------+-------------------+
| id|         name|create_date|   last_update_time|
+---+-------------+-----------+-------------------+
|  1|Chris Sharkey| 2020-01-01|2020-01-02 00:00:00|
|  2|         Will| 2020-01-01|2020-01-01 00:00:00|
|  3|         Emma| 2020-01-01|2020-01-01 00:00:00|
|  4|         John| 2020-01-01|2020-01-01 00:00:00|
|  5|         Eric| 2020-01-01|2020-01-01 00:00:00|
|  6|         Adam| 2020-01-01|2020-01-01 00:00:00|
|  7|        Kelly| 2020-01-02|2020-01-02 00:00:00|
+---+-------------+-----------+-------------------+

Lets do another upsert ...

In [10]:
# Create a new data frame with an updated last_update_time
data = [
        ("1", "Christopher Sharkey", "2020-01-01", datetime.strptime('2020-01-03 00:00:00', '%Y-%m-%d %H:%M:%S')),
        ("8", "Ella", "2020-01-03", datetime.strptime('2020-01-03 00:00:00', '%Y-%m-%d %H:%M:%S'))
]

schema = StructType([
        StructField("id", StringType(), False),
        StructField("name", StringType(), False),
        StructField("create_date", StringType(), False),             
        StructField("last_update_time", TimestampType(), False)
])

updateDF = spark.createDataFrame(data=data,schema=schema)

# updateDF.show()
# updateDF.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
# Upsert the records in updateDF
updateDF \
    .write \
    .format('org.apache.hudi') \
    .option('hoodie.datasource.write.operation', 'upsert') \
    .options(**hudiOptions) \
    .mode('append') \
    .save('s3://hudi-sharkech/myhudidataset/')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
# Check that the upsert worked
snapshotQueryDF = spark.read.format('org.apache.hudi').load('s3://hudi-sharkech/myhudidataset' + '/*/*')

# snapshotQueryDF.show()

snapshotQueryDF.select("id", "name", "create_date", "last_update_time").orderBy("id").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---+-------------------+-----------+-------------------+
| id|               name|create_date|   last_update_time|
+---+-------------------+-----------+-------------------+
|  1|Christopher Sharkey| 2020-01-01|2020-01-03 00:00:00|
|  2|               Will| 2020-01-01|2020-01-01 00:00:00|
|  3|               Emma| 2020-01-01|2020-01-01 00:00:00|
|  4|               John| 2020-01-01|2020-01-01 00:00:00|
|  5|               Eric| 2020-01-01|2020-01-01 00:00:00|
|  6|               Adam| 2020-01-01|2020-01-01 00:00:00|
|  7|              Kelly| 2020-01-02|2020-01-02 00:00:00|
|  8|               Ella| 2020-01-03|2020-01-03 00:00:00|
+---+-------------------+-----------+-------------------+

## Delete data

In [13]:
# Create a new data frame
data = [
        ("1", "Christopher Sharkey", "2020-01-01", datetime.strptime('2020-01-03 00:00:00', '%Y-%m-%d %H:%M:%S')),
        ("8", "Ella", "2020-01-03", datetime.strptime('2020-01-03 00:00:00', '%Y-%m-%d %H:%M:%S'))
]

schema = StructType([
        StructField("id", StringType(), False),
        StructField("name", StringType(), False),
        StructField("create_date", StringType(), False),             
        StructField("last_update_time", TimestampType(), False)
])

deleteDF = spark.createDataFrame(data=data,schema=schema)

# updateDF.show()
# updateDF.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
# Create hudiOptions variable
hudiOptionsDelete = {
    'hoodie.table.name': 'my_hudi_table',
    'hoodie.datasource.write.recordkey.field': 'id',
    'hoodie.datasource.write.partitionpath.field': 'create_date',
    'hoodie.datasource.write.precombine.field': 'last_update_time',
    'hoodie.datasource.write.operation': 'delete'
}

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
deleteDF \
    .write \
    .format('org.apache.hudi') \
    .options(**hudiOptionsDelete) \
    .mode('append') \
    .save('s3://hudi-sharkech/myhudidataset/')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
# Check that the upsert worked
snapshotQueryDF = spark.read.format('org.apache.hudi').load('s3://hudi-sharkech/myhudidataset' + '/*/*')

# snapshotQueryDF.show()

snapshotQueryDF.select("id", "name", "create_date", "last_update_time").orderBy("id").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---+-----+-----------+-------------------+
| id| name|create_date|   last_update_time|
+---+-----+-----------+-------------------+
|  2| Will| 2020-01-01|2020-01-01 00:00:00|
|  3| Emma| 2020-01-01|2020-01-01 00:00:00|
|  4| John| 2020-01-01|2020-01-01 00:00:00|
|  5| Eric| 2020-01-01|2020-01-01 00:00:00|
|  6| Adam| 2020-01-01|2020-01-01 00:00:00|
|  7|Kelly| 2020-01-02|2020-01-02 00:00:00|
+---+-----+-----------+-------------------+